**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.
/home/charles/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function ```act``` allows the agent to chose an action. If train is False, then that means we are testing the algorithm. Therefore, we want to use the policy we learned (using the Q function). So we play using the ```learned_act``` fonction. When train is true, we also want to explore. Therefore sometimes we select a random action using epsilon.

We need $\epsilon$ because we want to be able to efficiently explore the state space. If we were to always follow function Q, we could imagine starting from random Q-functions that would lead to being stuck everytime in some specific states, and thus we would not learn. With $\epsilon$, we are guaranteed that we are going to explore the state space, and by doing so, we will be able to update more accurtely the Q-function. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #I modified the following line from self.position[-2:, :] = -1 to 
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #I modified the following line from self.position[-2:, :] = -1 to 
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=11 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The aray ```board``` stores the rewards the rat will earn on each tile if it steps on it. It is initially set to 0 everywhere in the ```init``` function, but then it is randomized in the ```reset``` function. The ```position``` variable tracks the previous position of the rat when it acts, with the command ```self.position[self.x, self.y] = 1``` before the rat moves. Moreover, two tiles away from the border, the position is set to -1. That can help the rat know that it is near the border of the grid, as it is quite myopic and can't see very far. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Draw randromly an integer in the interval [0,self.n_action[.
        # The environment already takes care of not getting out of the grid, so we don't have to
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # Game over will happen because there is a max time
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action according to the policy learned (we are not training anything here)
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random10.mp4'))

Win/lose count 9.0/15.0. Average score (-6.0)
Win/lose count 11.5/11.0. Average score (-2.75)
Win/lose count 7.5/8.0. Average score (-2.0)
Win/lose count 6.5/5.0. Average score (-1.125)
Win/lose count 8.5/17.0. Average score (-2.6)
Win/lose count 13.5/18.0. Average score (-2.9166666666666665)
Win/lose count 6.0/21.0. Average score (-4.642857142857143)
Win/lose count 11.0/21.0. Average score (-5.3125)
Win/lose count 13.5/16.0. Average score (-5.0)
Win/lose count 10.5/16.0. Average score (-5.05)
Win/lose count 12.0/14.0. Average score (-4.7727272727272725)
Final score: -4.7727272727272725


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




-  Let's show that \begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
By definition, the Q function is \begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
We have that \begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \>  \\
=E_{p^{\pi}}[r(s_0, a_0) + \sum_{1\leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
=r(s, a) + \gamma E_{p^{\pi}}[\sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
=r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}}[\sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']] \\
=r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}[E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']] \text{  (just changing the notations)} \\
=r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')] \\
=E_{(s',a')\sim p(.|s,a)}[r(s, a) + \gamma Q^{\pi}(s',a')]
\end{equation*}
<br/>
<br/>
<br/>
<br/>



-  Let's show that \begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
By definition, we have that \begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}
Therefore,
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> \\
= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
= r(s,a)+\gamma \max_{\pi} E_{(s',a')\sim p(.|s,a)}[ Q^{\pi}(s',a')] \\
= r(s,a)+\gamma \max_{a'}\max_{\pi} E_{s'\sim \pi^*(.|s,a)}[ Q^{\pi}(s',a')] \\
= r(s,a)+\gamma \max_{a'}E_{s'\sim \pi^*(.|s,a)}[ Q^{*}(s',a')] \\
= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] 
\end{equation*}

We are able to switch expectation and $\max_{\pi}$, because $\pi$ is the policy that dictates the Q function for every possible outcome. We are able to switch esperance and $\max_{a'}$ using the Fubini theorem. The notation $s'\sim \pi^*(.|s,a)$ is not very good, because $s'$ is drawn by the environment, not by the policy. So when we write $s'\sim \pi^*(.|s,a)$  or $s'\sim p(.|s,a)$, that is in fact the same thing.
<br/>
<br/>
<br/>
<br/>


-  We would like to have the equation \begin{equation*}
Q(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q(s',a')].
\end{equation*} true for all $s$ and $a$. That way, we would have $Q= Q^*$. Therefore, we want to have  \begin{equation*}
E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q(s',a') - Q(s,a)].
\end{equation*} as close as 0 for all $s$ and $a$, which is the same as having 
\begin{equation*}
E_{s'\sim \pi^*(.|s,a)}[||r(s,a)+\gamma\max_{a'}Q(s',a') - Q(s,a)||^2].
\end{equation*}
as close to 0 as possible. Therefore, if we parametrize the Q function with $\theta$, 
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
seems like a plausible objective



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # If the memory is full, we delete the first element
        if len(self.memory) >= self.max_memory:
            del self.memory[0]
        # We add the new element at the end of the list
        self.memory.append(m)
            

    def random_access(self):
        # We randomly draw an index
        random_index = np.random.randint(len(self.memory))
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(-1, 5, 5, self.n_state)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        # We fill the following array with the matching information for every sample in a batch
        new_states = []
        rewards = []
        actions = []
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i,:,:,:] = s
            new_states.append(n_s)
            actions.append(a)
            rewards.append(r)

            # There should be no condition on game_over_ because this is an element of the
            # sample that we are adding to our dataset
            # Moreover, there shouldn't be any condition on game_over either because Q does not depend on the time
            # Therefore, if we introduce a different reward depending on time, we are effectively introducing
            # noise in the algorithm, and we don't want that. That is why I did not fill the lines below.
            # You can see that I still obtain very good results.
#             if game_over_:
#                 ######## FILL IN
#                 pass
#             else:
#                 ######## FILL IN
#                 pass
        ######## FILL IN
        # First we fill target_q with what we currently predict
        target_q = self.model.predict(input_states)
        next_q = self.model.predict(np.array(new_states))
        # Then we update the target with the actions we actually took using the formulas from question 5
        # Therefore, we don't change most coefficients of target_q by doing this
        target_q[np.arange(self.batch_size),np.array(actions)] = np.array(rewards) + \
                                                        self.discount * np.max(next_q, axis = 1)
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Sequential()
        # We whose a simple network with only one hidden layer, and relu activation, but it works very well
        model.add(Flatten())
        model.add(Dense(20, activation = "relu"))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [14]:
epochs_train = 101
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train100.mp4'))

Epoch 000/101 | Loss 0.0188 | Win/lose count 8.5/13.0 (-4.5)
Epoch 001/101 | Loss 0.0164 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/101 | Loss 0.0109 | Win/lose count 4.5/4.0 (0.5)
Epoch 003/101 | Loss 0.0098 | Win/lose count 7.5/9.0 (-1.5)
Epoch 004/101 | Loss 0.0092 | Win/lose count 6.5/5.0 (1.5)
Epoch 005/101 | Loss 0.0102 | Win/lose count 9.0/7.0 (2.0)
Epoch 006/101 | Loss 0.0188 | Win/lose count 6.5/5.0 (1.5)
Epoch 007/101 | Loss 0.0147 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/101 | Loss 0.0164 | Win/lose count 7.5/5.0 (2.5)
Epoch 009/101 | Loss 0.0092 | Win/lose count 6.0/8.0 (-2.0)
Epoch 010/101 | Loss 0.0072 | Win/lose count 7.0/4.0 (3.0)
Epoch 011/101 | Loss 0.0105 | Win/lose count 5.5/6.0 (-0.5)
Epoch 012/101 | Loss 0.0182 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/101 | Loss 0.0067 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/101 | Loss 0.0030 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/101 | Loss 0.0055 | Win/lose count 2.5/2.0 (0.5)
Epoch 016/101 | Loss 0.0051 | Win/lose count 5.0/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        # We use a network with 2 convolutional layers
        model.add(Conv2D(10, 3, padding = "same", activation = "relu"))
        model.add(Conv2D(10, 3, padding = "same", activation = "relu"))
        
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train100.mp4'))

Epoch 000/101 | Loss 0.0018 | Win/lose count 7.0/4.0 (3.0)
Epoch 001/101 | Loss 0.0039 | Win/lose count 7.5/1.0 (6.5)
Epoch 002/101 | Loss 0.0048 | Win/lose count 5.5/2.0 (3.5)
Epoch 003/101 | Loss 0.0009 | Win/lose count 4.0/1.0 (3.0)
Epoch 004/101 | Loss 0.0015 | Win/lose count 9.5/2.0 (7.5)
Epoch 005/101 | Loss 0.0020 | Win/lose count 6.0/4.0 (2.0)
Epoch 006/101 | Loss 0.0009 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/101 | Loss 0.0006 | Win/lose count 9.5/4.0 (5.5)
Epoch 008/101 | Loss 0.0078 | Win/lose count 7.5/2.0 (5.5)
Epoch 009/101 | Loss 0.0113 | Win/lose count 14.5/1.0 (13.5)
Epoch 010/101 | Loss 0.0008 | Win/lose count 13.0/2.0 (11.0)
Epoch 011/101 | Loss 0.0049 | Win/lose count 9.0/4.0 (5.0)
Epoch 012/101 | Loss 0.0011 | Win/lose count 4.5/1.0 (3.5)
Epoch 013/101 | Loss 0.0017 | Win/lose count 9.5/2.0 (7.5)
Epoch 014/101 | Loss 0.0065 | Win/lose count 2.5/2.0 (0.5)
Epoch 015/101 | Loss 0.0012 | Win/lose count 12.0/3.0 (9.0)
Epoch 016/101 | Loss 0.0020 | Win/lose count 12.5/4

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [25]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/0. Average score (10.0)
Win/lose count 16.0/0. Average score (13.0)
Win/lose count 4.0/0. Average score (10.0)
Win/lose count 8.0/0. Average score (9.5)
Win/lose count 10.5/0. Average score (9.7)
Win/lose count 4.0/0. Average score (8.75)
Win/lose count 3.5/0. Average score (8.0)
Win/lose count 11.5/0. Average score (8.4375)
Win/lose count 8.5/0. Average score (8.444444444444445)
Win/lose count 7.0/0. Average score (8.3)
Win/lose count 15.5/0. Average score (8.954545454545455)
Final score: 8.954545454545455
Test of the FC
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 3.0/0. Average score (2.25)
Win/lose count 7.0/0. Average score (3.8333333333333335)
Win/lose count 3.5/0. Average score (3.75)
Win/lose count 12.0/0. Average score (5.4)
Win/lose count 9.0/0. Average score (6.0)
Win/lose count 2.5/0. Average score (5.5)
Win/lose count 1.0/1.0. Average score (4.8125)
Win/lose count 0.5/0. Average score (4.333333333333333)
Win/lose count 2.5/0.

In [26]:
HTML(display_videos('cnn_test10.mp4'))

In [27]:
HTML(display_videos('fc_test10.mp4'))

We can see that we successfully trained both networks as they very rarely eat any poison, but eat some cheese. It seems that the CNN network achieves a better result than the FC network, wich makes sense as its architecture is adapted to the problem. However, when there are no more cheese near it, sometimes the mouse "runs in a loop". By that I mean that it stops exploring new states and comes and go back anfd forth on the same states whereas cheese is available further away. Sometimes, the mouse also makes mistakes. Some mistakes could probably be avoided with some more training, but others are a little bit more complicated. For example, when there is a wall of poison (at least 3 poisons in a row) blocking it from cheese, rather than trying to find a way around, it goes through the poison.  

When the temperature is higher, there is more cheese, so the mouse is able to eat more cheese before getting stuck. But it still gets stuck eventualy. When the temprature is lower, there is less cheese, so we get the worse score, and the mouse gets stuck pretty quickly. Also, when he temperature is low, we never eat any poison, but when the temprature is higher, we sometimes do. This is, in part, due to the effect mentionned above. But this also due to the fact that when there are more poisons on the map, we are likelier to eat them. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [28]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    epsilon = agent.epsilon

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(epsilon)
        counter = 0

        while not game_over:
            counter += 1
            # We update the epsilon
            agent.set_epsilon(agent.epsilon * (T - counter) / T)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # We added a malus on the position of the mouse so that is does not try to come back on the same state too much
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #I modified the following line from self.position[-2:, :] = -1 to 
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        reward = 0
        # We only give that reward when we train, because when we test, we want the true reward 
        # to be able to compare it to the initial method
        if train:
            reward = -self.malus_position[self.x, self.y]
        # We multiply the malus_position array by 0.9, th the mouse looses memory of where it was before
        # Other wise, the mouse can sometimes lock itself in a corner
        # This is a very effective method and increases the score by a lot
        self.malus_position *= 0.9
        # We add a malus on the current position of the mouse so it doesn't come back here again too much
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.t = self.t + 1
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # We need to add the malus_position state to the state, because the mouse needs to be able to see it
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over
    

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.malus_position[self.x, self.y] = 0.1


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        #I modified the following line from self.position[-2:, :] = -1 to 
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [30]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

Epoch 000/101 | Loss 0.0056 | Win/lose count 3.5/16.66589951845374 (-13.165899518453742)
Epoch 001/101 | Loss 0.0008 | Win/lose count 4.995245506651746/17.234425534973948 (-12.239180028322203)
Epoch 002/101 | Loss 0.0027 | Win/lose count 2.9999999999537117/19.459251740561882 (-16.45925174060817)
Epoch 003/101 | Loss 0.0029 | Win/lose count 2.4389234440409555/16.279569141351455 (-13.8406456973105)
Epoch 004/101 | Loss 0.0064 | Win/lose count 3.5/13.933506209786907 (-10.433506209786907)
Epoch 005/101 | Loss 0.0014 | Win/lose count 6.999999993395367/18.38315217592328 (-11.383152182527912)
Epoch 006/101 | Loss 0.0044 | Win/lose count 1.999753496519974/13.15169796077251 (-11.151944464252537)
Epoch 007/101 | Loss 0.0017 | Win/lose count 6.494985224840172/18.303500752414237 (-11.808515527574066)
Epoch 008/101 | Loss 0.0024 | Win/lose count 3.4771232070143814/16.507433947366426 (-13.030310740352045)
Epoch 009/101 | Loss 0.0065 | Win/lose count 4.458593004940102/8.007256507863334 (-3.5486635029

Epoch 081/101 | Loss 0.0029 | Win/lose count 24.49938794605091/6.287749229754659 (18.21163871629625)
Epoch 082/101 | Loss 0.0063 | Win/lose count 17.485460704790704/3.2771221599063693 (14.208338544884334)
Epoch 083/101 | Loss 0.0067 | Win/lose count 6.379691583269809/14.617400466890714 (-8.237708883620906)
Epoch 084/101 | Loss 0.0049 | Win/lose count 23.49721466768446/2.424783425086343 (21.072431242598118)
Epoch 085/101 | Loss 0.0042 | Win/lose count 16.499555659502814/6.79173438110306 (9.707821278399754)
Epoch 086/101 | Loss 0.0044 | Win/lose count 21.499997947072615/2.592839423567914 (18.9071585235047)
Epoch 087/101 | Loss 0.0047 | Win/lose count 19.9995107468594/5.01836607217599 (14.98114467468341)
Epoch 088/101 | Loss 0.0032 | Win/lose count 15.992368760720286/8.797182014189708 (7.195186746530577)
Epoch 089/101 | Loss 0.0054 | Win/lose count 18.999500624081122/7.3032353450757 (11.696265279005422)
Epoch 090/101 | Loss 0.0043 | Win/lose count 18.97168868922734/2.12285233391034 (16.84

In [31]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 22.0/0. Average score (22.0)
Win/lose count 22.0/1.0. Average score (21.5)
Win/lose count 12.5/0. Average score (18.5)
Win/lose count 18.5/0. Average score (18.5)
Win/lose count 21.0/0. Average score (19.0)
Win/lose count 17.0/1.0. Average score (18.5)
Win/lose count 13.5/1.0. Average score (17.642857142857142)
Win/lose count 27.5/0. Average score (18.875)
Win/lose count 21.0/0. Average score (19.11111111111111)
Win/lose count 18.5/1.0. Average score (18.95)
Win/lose count 17.5/0. Average score (18.818181818181817)
Final score: 18.818181818181817


The decreasong epsilon-greedy allows us to have a better training. However I don't think it encourages exploration during test. Indeed, we can see that with our previous models the mouse was performing near-optimal actions. That means that our Q function was quite close to $Q^*$. Changing epsilon can only affect our rate of convergence. Hence even if we get closer to $Q^*$, $Q^*$ can only tell the mouse to go towards a cheese when it sees it. But it is very myopic ! Hence, even with a better Q funciton, the mouse will still get stuck somewhere and stop exploring at some point during the test.

We can see that we obtain a MUCH better score than before thanks to the new state that we happened. We also added a "memory effect" where the mouse forgets overtime the states it was at (we multiply the new state by 0.9 every iteration), which makes exploring much more efficient (it cannot get stuck in a corner). And the mouse doesn't get stuck anymore as it used to. However, the mouse isn't always able to explore the whole board. It can get stuck in one half of the board. But this is still much better than before. However, the mouse also tends to eat more poison than before. That is because of the new malus, it sometimes feel like is is "less bad" to eat a poison compared to before.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.